In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np

data4 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/sqsq.csv")
data4 = data4.drop(data4[data4.Label == 'Label'].index)

NORM_LEN = 2000
DDOS_LEN = 2000
BRUTE_LEN = 1500
SQL_LEN = 2500
data1 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/norm.csv")
data2 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/ddos.csv")
data3 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/nb1.csv")
data1 = data1.sample(NORM_LEN)
data2 = data2.sample(DDOS_LEN)
data3 = data3.sample(BRUTE_LEN)
df = data4.sample(SQL_LEN)
df = df.loc[df['Label'] == 'Infilteration']
df2 = data4.sample(1500)
data = pd.concat([data1, data2, data3, df, df2], ignore_index=True, sort=False)
shuffle = data.sample(frac=1, random_state=101).reset_index()

<ipython-input-4-9c976e5d1777>:4: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  data4 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/sqsq.csv")


In [5]:
shuffle.fillna(0, inplace=True)
df = shuffle

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
test_df = df.copy()

In [8]:
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_df.dropna(how="all", inplace=True)

number_inf = test_df[test_df == np.inf].count()

In [9]:
def ip2int(ip):
    if ip == 0:
      ip = '0.0.0.0'
    ip_list = list(map(int, ip.split('.')))
    num = 0
    for i in ip_list:
        num *= 256
        num += i
    return num

In [10]:
def delete_first_zero(val):
    if val[0] == '0':
        return val[1:]
    return val

def time2int(tm):
    tm_split = tm.split(' ')
    day_info = tm_split[0].split('/')
    day_info = list(map(delete_first_zero, day_info))
    day_info = list(map(int, day_info))
    day_amount = day_info[0] + (day_info[1] - 1) * 30 + (day_info[2] - 1970) * 365
    
    time_info = tm_split[1].split(':')
    time_info = list(map(delete_first_zero, time_info))
    time_info = list(map(int, time_info))
    sec_amount = 60 * (time_info[0] * 60 + time_info[1]) + time_info[2]
    
    if len(tm_split) == 3 and tm_split[-1] == 'PM':
        sec_amount += 3600 * 12
    sec_amount += day_amount * 24 * 3600
    
    return sec_amount

In [11]:
y = test_df['Label']
test_df = test_df.drop(['Label'], axis=1)
test_df['Src IP'] = test_df['Src IP'].apply(ip2int)
test_df['Dst IP'] = test_df['Dst IP'].apply(ip2int)
test_df['Timestamp'] = test_df['Timestamp'].apply(time2int)
test_df.head()

,index,Unnamed: 0,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,3381,76679.0,192.168.2.109-203.73.24.75-4018-80-6,3232236141,4018.0,3410565195,80,6,1275491536,199,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10050.251256,199.0,0.0,199.0,199.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,20,20,5025.125628,5025.125628,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,1,0,-1,63610,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4737,0.0,0,0,0.0,0,21,6,1517569997,3,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,666666.666667,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,40,20,333333.333333,333333.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,26883.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4050,0.0,0,0,0.0,0,21,6,1517570060,20,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,100000.0,20.0,0.0,20,20,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,40,20,50000.0,50000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,26883.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5990,0.0,0,0,0.0,0,0,0,1519035073,119792241,65,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.542606,1871753.766,2900390.549,17800000,515564,120000000,1871753.766,2900390.549,17800000,515564,0,0.0,0.0,0,0,0,0,0,0,0,0,0.542606,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,65,0,0,0,-1,-1,0,0,79000000.0,0.0,79000000,79000000,16800000.0,1454076.0,17800000,15800000
4,7640,0.0,0,0,0.0,0,53,17,1519006073,338,1,1,28,44,28,28,28.0,0.0,44,44,44.0,0.0,213017.7515,5917.159763,338.0,0.0,338,338,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,8,8,2958.579882,2958.579882,28,44,33.333333,9.237604,85.333333,0,0,0,0,0,0,0,0,1,50.0,28.0,44.0,0,0,0,0,0,0,1,28,1,44,-1,-1,0,8,0.0,0.0,0,0,0.0,0.0,0,0


In [12]:
IMPORTANT_COLS = ['Src IP',
                  'Dst IP',
                  'Timestamp',
                  'Src Port',
                  'Dst Port',
                  'Protocol','Flow Duration',
                  'Flow Byts/s',
                  'Flow Pkts/s',
                  'Pkt Len Var',
                  'Down/Up Ratio'
                 ]

In [13]:
test_df = test_df.drop([i for i in list(test_df.columns) if i not in IMPORTANT_COLS], axis=1)
test_df.head()

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Flow Byts/s,Flow Pkts/s,Pkt Len Var,Down/Up Ratio
0,3232236141,4018.0,3410565195,80,6,1275491536,199,0.0,10050.251256,0.0,1.0
1,0,0.0,0,21,6,1517569997,3,0.0,666666.666667,0.0,1.0
2,0,0.0,0,21,6,1517570060,20,0.0,100000.0,0.0,1.0
3,0,0.0,0,0,0,1519035073,119792241,0.0,0.542606,0.0,0
4,0,0.0,0,53,17,1519006073,338,213017.7515,5917.159763,85.333333,1


In [14]:
test_df = test_df.fillna(0)

In [15]:
test_df = test_df.astype(float)

In [16]:
df_z_scaled = test_df.copy()
  
# apply normalization techniques
for column in df_z_scaled.columns:
    df_z_scaled[column] = (df_z_scaled[column] -
                           df_z_scaled[column].mean()) / df_z_scaled[column].std()
df_z_scaled = df_z_scaled.fillna(0)
df_z_scaled.head()

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Flow Byts/s,Flow Pkts/s,Pkt Len Var,Down/Up Ratio
0,1.220203,-0.594858,1.573614,-0.322070,-0.454620,-1.909747,-0.407170,0.0,0.0,-0.349169,0.125365
1,-0.919581,-0.757115,-0.909605,-0.326343,-0.454620,0.520496,-0.407177,0.0,0.0,-0.349169,0.125365
2,-0.919581,-0.757115,-0.909605,-0.326343,-0.454620,0.520497,-0.407177,0.0,0.0,-0.349169,0.125365
3,-0.919581,-0.757115,-0.909605,-0.327863,-1.834592,0.535204,3.724508,0.0,0.0,-0.349169,-0.505610
4,-0.919581,-0.757115,-0.909605,-0.324025,2.075329,0.534913,-0.407166,0.0,0.0,-0.348570,0.125365


In [17]:
y = y.str.replace('ddos', '1').str.replace('Benign','0').str.replace('FTP-BruteForce','2').str.replace('Infilteration','3')
y = y.astype(np.int64)

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_z_scaled, y, test_size=0.3, random_state=42)

In [19]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.4 MB/s eta 0:00:00


In [22]:
from catboost import CatBoostClassifier

catboost = CatBoostClassifier()
params = {
"n_estimators": [10, 50, 200],
"learning_rate": [0.001, 0.01, 0.1, 0.5],
"max_depth": [4, 7],
"l2_leaf_reg": [0.1, 0.3, 0.5],
}
params = {
"n_estimators": [50],
"learning_rate": [0.5],
"max_depth": [7],
"l2_leaf_reg": [0.5],
}
catboost.grid_search(params, X_train, y_train, cv=5, plot=True, refit=True)
print(catboost.best_score_)
best_params = catboost.get_params()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.5667621	test: 0.5703529	best: 0.5703529 (0)	total: 68.3ms	remaining: 3.35s
1:	learn: 0.3308166	test: 0.3271174	best: 0.3271174 (1)	total: 84.2ms	remaining: 2.02s
2:	learn: 0.2422980	test: 0.2411434	best: 0.2411434 (2)	total: 100ms	remaining: 1.57s
3:	learn: 0.1734329	test: 0.1692309	best: 0.1692309 (3)	total: 116ms	remaining: 1.33s
4:	learn: 0.1344997	test: 0.1280437	best: 0.1280437 (4)	total: 133ms	remaining: 1.2s
5:	learn: 0.1154322	test: 0.1077916	best: 0.1077916 (5)	total: 149ms	remaining: 1.09s
6:	learn: 0.1021886	test: 0.0933223	best: 0.0933223 (6)	total: 164ms	remaining: 1.01s
7:	learn: 0.0910718	test: 0.0815475	best: 0.0815475 (7)	total: 179ms	remaining: 940ms
8:	learn: 0.0853029	test: 0.0777824	best: 0.0777824 (8)	total: 195ms	remaining: 890ms
9:	learn: 0.0807856	test: 0.0747842	best: 0.0747842 (9)	total: 213ms	remaining: 852ms
10:	learn: 0.0774374	test: 0.0735806	best: 0.0735806 (10)	total: 229ms	remaining: 813ms
11:	learn: 0.0735057	test: 0.0717895	best: 0.071789

In [23]:
best_params

{'depth': 7, 'iterations': 50, 'learning_rate': 0.5, 'l2_leaf_reg': 0.5}

In [24]:
y_pred = catboost.predict(X_test)

In [26]:
from sklearn.metrics import classification_report, accuracy_score
print('Results on the test set:')
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test,y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       921
           1       0.99      1.00      1.00       591
           2       1.00      1.00      1.00       477
           3       0.91      0.93      0.92       337

    accuracy                           0.97      2326
   macro avg       0.97      0.97      0.97      2326
weighted avg       0.97      0.97      0.97      2326

0.9742046431642304


Results on the test set:
              precision    recall  f1-score   support

      Normal       0.97      0.96      0.96       642
        DdoS       0.98      0.99      0.99       163
 Brute-focre       1.00      1.00      1.00       289 SQL-injection     0.78      0.83      0.80       106

    accuracy                           0.96      1200
   macro avg       0.93      0.95      0.94      1200
weighted avg       0.96      0.96      0.96      1200